In [1]:
import icechunk
import zarr

Lets create an in-memory icechunk store

In [2]:
store = await icechunk.IcechunkStore.from_json(config={"storage": {"type": "in_memory"}, "dataset": {}}, mode="w")
store

Ok! Lets create some data!

In [3]:
group = zarr.group(store=store, overwrite=True)
group

Group(_async_group=<AsyncGroup <icechunk.IcechunkStore object at 0x1034f6e90>>)

In [4]:
air_temp = group.create_array("air_temp", shape=(1000, 1000), chunk_shape=(100, 100), dtype="i4")
air_temp

<Array <icechunk.IcechunkStore object at 0x1034f6e90>/air_temp shape=(1000, 1000) dtype=int32>

In [5]:
async for key in store.list():
    print(key)

zarr.json
air_temp/zarr.json


In [6]:
air_temp[:, :] = 42

In [7]:
air_temp[200, 6]

array(42, dtype=int32)

Now that we have set the values, lets commit

In [8]:
snapshot_id = await store.commit("main", "Initial commit")
snapshot_id

'fbf6189b1e3fc175da52d93987de038a'

Okay now we can change the data

In [9]:
air_temp[:, :] = 54

In [10]:
air_temp[200, 6]

array(54, dtype=int32)

And we can commit again

In [11]:
new_snapshot_id = await store.commit("main", "Change air temp to 54")
new_snapshot_id

'a6707edbc0667661025a9947aee42b13'

Cool, now lets checkout the original snapshot and see if the value is 42 again

In [13]:
await store.checkout(snapshot_id=snapshot_id)
air_temp[200, 6]

array(42, dtype=int32)